In [ ]:
class Normalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, image, target=None):
        image = F.normalize(image, mean=self.mean, std=self.std)
        if target is None:
            return image, None
        target = target.copy()
        h, w = image.shape[-2:]
        if "boxes" in target:
            boxes = target["boxes"]
            boxes = box_xyxy_to_cxcywh(boxes)
            boxes = boxes / torch.tensor([w, h, w, h], dtype=torch.float32)
            target["boxes"] = boxes
        return image, target


def box_xyxy_to_cxcywh(x):
    x0, y0, x1, y1 = x.unbind(-1)
    b = [(x0 + x1) / 2, (y0 + y1) / 2,
         (x1 - x0), (y1 - y0)]
    return torch.stack(b, dim=-1)

    
img, target = Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(img, target)

# 代码解读 ：
# 将图像的像素值进行归一化
# 将 bbox 坐标由 (xmin, ymin, xmax, ymax)  转换为 （cx, cy, w, h）的形式
# 将 （cx, cy, w, h）的取值分别除以（wimg ,himg ,wimg ,himg ），将坐标值都归一化到 0 ～1 之间，
# 即，相对于 图像尺寸的 相对坐标